Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [2]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = 'C:/Users/ryanh/DS-Unit-2-Linear-Models/data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [3]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [28]:
# set df to read all columns and inspect head 

pd.set_option('display.max_columns', None)
df.head(3)

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,description_length,total_perks,amenities
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,588,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,4,4
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,691,3,3


In [6]:
# convert to datetime 

df['created'] = df['created'].apply(np.datetime64)

In [29]:
# create description length and amenities

df['description_length'] = df['description'].apply(lambda s: len(s) if isinstance(s, str) else 0)
df['amenities'] = df['cats_allowed'] + df['hardwood_floors'] + df['dogs_allowed'] + df['doorman'] + df['dishwasher'] + df['laundry_in_building'] + df['fitness_center'] +  df['laundry_in_unit'] + df['roof_deck'] + df['outdoor_space'] + df['dining_room'] + df['high_speed_internet'] + df['balcony'] + df['swimming_pool'] + df['terrace'] + df['loft'] + df['garden_patio'] + df['wheelchair_access'] + df['common_outdoor_space']

In [30]:
# create test and train data 

train_date1 = np.datetime64("2016-03-01")
train_date2 = np.datetime64("2016-05-01")

test_date1 = np.datetime64("2016-05-01")
test_date2 = np.datetime64("2016-06-01")

train = df[(train_date1 <= df["created"]) & (df["created"] < train_date2)]
test = df[(test_date1 <= df["created"]) & (df["created"] < test_date2)]

In [31]:
# imports 

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# create features and targets 

features = ["description_length", "amenities"]
target = "price"

# create model and fit model based on features and targets 

model = LinearRegression()
model.fit(train[features], train[target])

# mae

mean_absolute_error(model.predict(test[features]), test[target])

1150.235158544352

In [32]:
# model's coefficients and intercept 

coef = model.coef_
intercept = model.intercept_

print(f"coef: {coef}, intercepts: {intercept}")

coef: [  0.22474988 170.44238904], intercepts: 2807.9134712092387


In [33]:
# metrics for test data

predict = model.predict(test[features])
actual = test[target]

mae = mean_absolute_error(predict, actual)
R2 = r2_score(predict, actual)
RMSE = mean_squared_error(predict, actual)**0.5

print(f"""
    Metrics for test data:
    MAE: {mae}
    R2: {R2}
    RMSE: {RMSE}
""")


    Metrics for test data:
    MAE: 1150.235158544352
    R2: -9.436413980232002
    RMSE: 1707.6233776244271



In [34]:
# metrics for train data 

predict = model.predict(train[features])
actual = train[target]

mae = mean_absolute_error(predict, actual)
R2 = r2_score(predict, actual)
RMSE = mean_squared_error(predict, actual)**0.5

print(f"""
    Metrics for the train data:
    MAE: {mae}
    R2: {R2}
    RMSE: {RMSE}
""")


    Metrics for the train data:
    MAE: 1111.834387754279
    R2: -8.878323198547946
    RMSE: 1645.9371204245085

